In [4]:
from lda2vec import preprocess, Corpus
import numpy as np

You must be using a very recent version of pyLDAvis to use the lda2vec outputs. 
As of this writing, anything past Jan 6 2016 or this commit `14e7b5f60d8360eb84969ff08a1b77b365a5878e` should work.
You can do this quickly by installing it directly from master like so:


In [ ]:
# pip install git+https://github.com/bmabey/pyLDAvis.git@master#egg=pyLDAvis

In [11]:
import pyLDAvis
pyLDAvis.enable_notebook()

### Reading in the saved model topics

After runnning `lda2vec_run.py` script in `examples/twenty_newsgroups/lda2vec` directory a `topics.pyldavis.npz` will be created that contains the topic-to-word probabilities and frequencies. What's left is to visualize and label each topic from the it's prevalent words.

In [63]:
npz = np.load(open('topics.pyldavis.npz', 'r'))
dat = {k: v for (k, v) in npz.iteritems()}
dat['vocab'] = dat['vocab'].tolist()
dat['term_frequency'] /= dat['term_frequency'].sum()

In [64]:
top_n = 10
for j, topic_to_word in enumerate(dat['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    msg += ' '.join([dat['vocab'][i].strip()[:35] for i in top])
    print msg

Topic 0 server xterm compile widget colormap gc client x xdm x11r5
Topic 1 jesus mary god christ father her she matthew karina prophets
Topic 2 government morality clipper chip secret nsa decrypt keys homicides israel self-defense
Topic 3 game games bike braves team scored baseball season alomar playoffs
Topic 4 njd 5 - calgary det 3 winnipeg 6 1 hartford
Topic 5 the united states privacy anonymity turks turkey eff greece firearms adl militia
Topic 6 hiv launched bullock venus satellite magi committee mission hicnet medical newsletter page erzurum
Topic 7 nsa government encryption clipper chip chip rsa keys clipper decrypt escrow
Topic 8 screen windows dos controller power supply printer cable fonts scsi disk
Topic 9 apartment sumgait karina went had was scored agdam baku he
Topic 10 jesus x god widget matthew faith scripture bible colormap the holy spirit
Topic 11 god christian faith atheists christianity jesus bible morality belief homosexuality
Topic 12 jesus god christ christian sh

### Visualize topics

In [70]:
len(dat['vocab'])

4105

In [73]:
dat['topic_term_dists'][:, 0]

array([ 0.00018633,  0.00019031,  0.00020824,  0.00019813,  0.00020441,
        0.00025016,  0.000226  ,  0.00020229,  0.00020269,  0.00019499,
        0.00021055,  0.00018559,  0.00018576,  0.00019086,  0.0001877 ,
        0.00020213,  0.00021035,  0.00018432,  0.00022138,  0.00019041], dtype=float32)

In [65]:
import warnings
warnings.filterwarnings('ignore')
prepared_data = pyLDAvis.prepare(dat['topic_term_dists'], dat['doc_topic_dists'], 
                                 dat['doc_lengths'], dat['vocab'], dat['term_frequency'], mds='tsne')

In [66]:
pyLDAvis.display(prepared_data)

### 'True' topics

The 20 newsgroups dataset is interesting because users effetively classify the topics by posting to a particular newsgroup. This lets us qualitatively check our unsupervised topics with the 'true' labels. For example, the four topics we highlighted above are intuitively close to comp.graphics, sci.med, talk.politics.misc, and sci.space.

    comp.graphics
    comp.os.ms-windows.misc
    comp.sys.ibm.pc.hardware
    comp.sys.mac.hardware
    comp.windows.x    
    rec.autos
    rec.motorcycles
    rec.sport.baseball
    rec.sport.hockey    
    sci.crypt
    sci.electronics
    sci.med
    sci.space
    misc.forsale    
    talk.politics.misc
    talk.politics.guns
    talk.politics.mideast    
    talk.religion.misc
    alt.atheism
    soc.religion.christian